### Preparation

In [1]:
#!pip install numpy
#!pip install pandas
#!pip install requests
#!pip install tqdm
#!pip install sqlalchemy

### Packages

In [2]:
import numpy as np
import pandas as pd
import requests
import os
from tqdm import tqdm
from sqlalchemy import create_engine

### Input

In [3]:
stocks    = ('AAPL', 'IBM', 'BABA', 'AMZN', 'META') #max 5 stocks for free API key

key       = '' #put your API key from https://www.alphavantage.co/support/#api-key
capital   = 100 #$
UTC       = 7
SMA       = 28 #Simple Moving Average
timeframe = '60min'
#tradingfee = 0

### Download data

In [4]:
def download_data(symbol, timeframe):
    url = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=' + symbol + '&interval=' + timeframe + '&outputsize=full&apikey=' + key
    
    resp = requests.get('https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=' + symbol + '&interval=' + timeframe + '&outputsize=full&apikey=' + key)
    frame = pd.DataFrame(resp.json()['Time Series (' + timeframe + ')']).T.loc[::-1]
    frame.columns = ['Open', 'High', 'Low','Close','Volume']
    frame[['Open', 'High', 'Low', 'Close']] = frame[['Open', 'High', 'Low', 'Close']].astype(float)
    frame.index = pd.to_datetime(frame.index) + pd.Timedelta(hours=UTC)
    frame = frame.reset_index().rename(columns={'index': 'Time'})
    
    return frame

In [5]:
if os.path.exists("Stock.db"):
    os.remove("Stock.db")
    
engine = create_engine('sqlite:///Stock.db')

for stock in tqdm(stocks):
    download_data(stock, timeframe).to_sql(stock, engine, index=False)

100%|██████████| 5/5 [00:06<00:00,  1.21s/it]


### Main Program

In [6]:
class Backtest_SMA:
    
    def __init__(self, capital, SMA, stocks): 
        self.capital  = capital
        self.SMA      = SMA
        self.stocks   = stocks
            
    def method(self, stock):
        data = pd.read_sql(stock, engine).set_index('Time') 
        
        data['sma'] = data.Close.rolling(self.SMA).mean()
        data.dropna(inplace=True)
    
        data['Type'] = np.where(data['Close'].shift(1) > data['sma'].shift(1), 'long', 'short')
        
        data['change'] = data.Close.pct_change()
        
    
        data['long strategy']  = np.where(data['Close'] > data['sma'], 1, 0)
        data['Long positions'] = self.capital*(np.cumprod(data['long strategy'].shift(1)*data['change'] + 1))
        
        data['short strategy']  = np.where(data['Close'] > data['sma'], 0, -1)
        data['Short positions'] = self.capital*(np.cumprod(data['short strategy'].shift(1)*data['change'] + 1))
        
        data['Return SMA'] = data['Long positions']*data['Short positions']/self.capital
        data['Return hold']  = self.capital*np.cumprod(data['change']+1)
    
        data['win pos'] = np.where(data['Close']> data['sma'], 1, -1)
        data['win']     = np.cumsum(np.where(data['win pos'].shift(1)*data['change'] > 0, 1, 0))
        
        data.dropna(inplace=True)
        data['Winrate'] = data['win']/np.arange(1, len(data) + 1)
        
        return data.loc[:, ['Winrate', 'Type', 'Long positions', 'Short positions', 'Return SMA', 'Return hold']]
    
    
    def sorting(self):
        profits = []
        
        for stock in self.stocks:
            df = self.method(stock)
            
            ret = df[['Return SMA', 'Return hold']]
            ret.columns = ['Return SMA ($)', 'Return hold ($)']
            
            profits.append(ret.iloc[-1,:])

        frame = pd.DataFrame(profits, self.stocks)
        frame['edges ($)'] = frame['Return SMA ($)'] - frame['Return hold ($)']
        frame = frame.sort_values('edges ($)', ascending=False)
        return frame

    
    def output(self, stock):
        data = self.method(stock)
        
        print('Simulation of ' + '\033[1m' + str(stock) + '\033[0m' + ' runs from ' + '\033[1m' + str(data.index[0]) + '\033[0m' + ' to ' + '\033[1m' + str(data.index[-1]) + '\033[0m')
        print('Winrate Method: %5.2f' %(100*data['Winrate'][-1]) + '%\n')

        print('Return Hold                : $%5.2f' %(data['Return hold'][-1]))
        print('return from Long Positions : $%5.2f' %(data['Long positions'][-1]))
        print('return from Short Positions: $%5.2f' %(data['Short positions'][-1]))
        print('Return Method              : $%5.2f' %(data['Return SMA'][-1]))

        
    def journal(self, stock):
        if os.path.exists('Journal ' + stock + '.xlsx'):
            os.remove('Journal ' + stock + '.xlsx')
        
        self.method(stock).to_excel('Journal ' + stock + '.xlsx', engine='xlsxwriter') 

In [7]:
backtesting = Backtest_SMA(capital, SMA, stocks)

### Output

In [8]:
#backtesting.method('META')

In [9]:
backtesting.sorting()

,Return SMA ($),Return hold ($),edges ($)
META,118.804826,79.319280,39.485546
AMZN,89.414606,83.209687,6.204919
AAPL,83.332708,97.673546,-14.340838
IBM,95.944877,118.281981,-22.337105
BABA,67.077244,97.932145,-30.854901


In [14]:
backtesting.output('META')

Simulation of META runs from 2022-09-23 00:00:00 to 2022-11-17 03:00:00
Winrate Method: 47.77%

Return Hold                : $79.32
return from Long Positions : $99.29
return from Short Positions: $119.66
Return Method              : $118.80


In [15]:
backtesting.journal('META')